In [18]:
# 모델과 문장 분리를 위한 토크나이저를 불러온다.
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import torch
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [98]:
text = """
보려고요 사장님도 친절하시구 내부 분위기도 모던하고 앤틱감성이 있어서
석계에 이렇게 분위기 좋은 곳이 있는지
일단 인테리어가 너무 예쁘고 조용한
분위기라 가족모임으로 넘 좋더라구요
분위기도 너무 좋고 무엇보다 파스타가 진짜 맛도리예요 청양크림 무조건 시키시고
분위기좋은 곳에서 오순도순 데이트하셔요
석계역 맛집으로 유명한 라체나 1907이예요 분위기도 좋고
인테리어도 아기자기하니 분위기가 좋았어요
분위기 깡패인 석계역맛집
맛 부터 인테리어까지 다 너무 만족하면서 먹고왔어요
음식도 맛있고 분위기도 넘 좋았어요
구운 야채랑 소스들이랑 궁합이 좋습니다 분위기도 좋아서
음식 맛있고 매장 분위기도 아늑하고
분의기도 괜찮고 시설도 깔끔해요
분위기 넘 좋고 맛은 또 말모말모
저녁 시간에 분위기가 더 좋을 듯 한데 아이 데리고는 힘들듯
"""

In [99]:
# 개행 문자 기준으로 분리하고 각 줄의 앞뒤 공백 제거
lines = [line.strip() for line in text.strip().split('\n') if line.strip()]

# ". "로 구분하여 한 문자열로 합치기
text = ". ".join(lines) + "."

# 결과 출력
print(text)

보려고요 사장님도 친절하시구 내부 분위기도 모던하고 앤틱감성이 있어서. 석계에 이렇게 분위기 좋은 곳이 있는지. 일단 인테리어가 너무 예쁘고 조용한. 분위기라 가족모임으로 넘 좋더라구요. 분위기도 너무 좋고 무엇보다 파스타가 진짜 맛도리예요 청양크림 무조건 시키시고. 분위기좋은 곳에서 오순도순 데이트하셔요. 석계역 맛집으로 유명한 라체나 1907이예요 분위기도 좋고. 인테리어도 아기자기하니 분위기가 좋았어요. 분위기 깡패인 석계역맛집. 맛 부터 인테리어까지 다 너무 만족하면서 먹고왔어요. 음식도 맛있고 분위기도 넘 좋았어요. 구운 야채랑 소스들이랑 궁합이 좋습니다 분위기도 좋아서. 음식 맛있고 매장 분위기도 아늑하고. 분의기도 괜찮고 시설도 깔끔해요. 분위기 넘 좋고 맛은 또 말모말모. 저녁 시간에 분위기가 더 좋을 듯 한데 아이 데리고는 힘들듯.


In [100]:
# 토크나이저를 사용하여 뉴스기사 원문을 모델이 인식할 수 있는 토큰 형태로 바꿔주기
input_ids = tokenizer.encode(text)

In [101]:
# 모델에 넣기 전 문장의 시작과 끝을 나타내는 토큰을 추가하기
import torch
input_ids = [tokenizer.bos_token_id] + input_ids + [tokenizer.eos_token_id]
input_ids = torch.tensor([input_ids])

In [105]:
summary_text_ids = model.generate(
    input_ids = input_ids,
    bos_token_id = model.config.bos_token_id,
    eos_token_id = model.config.eos_token_id,
    length_penalty= 1, # 길이에 대한 penalty 값. 1보다 작은 경우 더 짧은 문장을 생성하도록 유도하며, 1보다 클 경우 길이가 더 긴 문장을 유도
    max_length = 150, # 요약문의 최대 길이 설정
    min_length = 50, # 요약문의 최소 길이 설정
    num_beams = 5) # 문장 생성 시 다음 단어를 탐색하는 영역의 개수

In [106]:
# 요약문 출력 : 왜 긴 공백이 생기는지는 모르겠음
summary = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

In [107]:
cleaned_text = "\n".join([line for line in summary.splitlines() if line.strip()])

print(cleaned_text)

석계역 맛집으로 유명한 라체나 1907은 인테리어가 너무 예쁘고 조용한 분위기라 가족모임으로 넘 좋았고 석계역 맛집으로 유명한 라체나 1907과 인테리어가 아기자기한 분위기라 오순도순 데이트하셔요.


# 각 카테고리의 문장들을 요약할 때 주의할 점
위의 taste_text문장처럼 끊어진 문장들을 개행없이 한줄로 이어줘야함. + 한 문장들 사이사이느 '.'으로 구분해줘야함. => 개행없이 마침표로 구분.

In [ ]:
ㅌ